In [75]:
import pandas as pd  # 판다스 불러오기

# 그냥 불러오면 행 데이터 생략이 많아서 모든 행과 열 전체 데이터가 나오도록 판다스 설정 변경해주기
pd.set_option('display.max_rows',None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_colwidth', None)  # 열 너비 제한 없애기
pd.set_option('display.width', None)  # 줄바꿈 없애기

# 엑셀파일 다운로드 후, 경로 설정과 데이터 불러오기
file_path = '/Users/leehyeon/Downloads/관서별 5대범죄 발생 및 검거.xlsx'
df = pd.read_excel(file_path)  # 윗줄처럼 따로 경로 설정 없이 바로 ()안에 해당 경로를 적어줄 수도 있음

# Series 를 활용하여 기존 DataFrame 에 column 추가하기
ad = pd.Series(['서대문구', '강남구', '강서구', '서초구', '은평구', '중구', '종로구', '용산구', '성북구', '동대문구',
             '마포구', '영등포구', '성동구', '동작구', '광진구', '강북구', '금천구', '중랑구', '관악구', '강동구',
             '구로구', '양천구', '송파구', '노원구', '서초구', '은평구', '도봉구'])  # Series 는 함수 호출 방식으로, ()를 사용해야하기 때문에 list 형태의 데이터를 만들었다면 꼭 ()로 묶어줘야함
df['구 별'] = ad  # '구 별' 이라는 column 이 추가됨

# map 과 딕셔너리를 사용하여 관서명을 구 이름에 매핑하기(map 이 이런식의 간단한 데이터 매칭에 활용하기 좋다)
area_map = {'서대문서': '서대문구', '수서서': '강남구', '강서서': '강서구', '서초서': '서초구',
            '서부서': '은평구', '중부서': '중구', '종로서': '종로구', '남대문서': '중구',
            '혜화서': '종로구', '용산서': '용산구', '성북서': '성북구', '동대문서': '동대문구',
            '마포서': '마포구', '영등포서': '영등포구', '성동서': '성동구', '동작서': '동작구',
            '광진서': '광진구', '강북서': '강북구', '금천서': '금천구', '중랑서': '중랑구',
            '강남서': '강남구', '관악서': '관악구', '강동서': '강동구', '종암서': '성북구',
            '구로서': '구로구', '양천서': '양천구', '송파서': '송파구', '노원서': '노원구',
            '방배서': '서초구', '은평서': '은평구', '도봉서': '도봉구'}  # area_map 이라는 변수에 딕셔너리 형태의 데이터 할당
df['구 별'] = df['관서명'].map(area_map)  # 할당해준 변수를 활용하여 map 으로 '구 별' column 의 데이터 값(딕셔너리 키값)을 '관서명' column 데이터값(딕셔너리 밸류값)에 맞게 매칭시켜줌

# fillna 를 사용하여 NaN 값 대체하기
df = df.fillna('구 없음')  # fillna 로 특정 열의 값만 대체하거나, 한 개가 아닌 여러 값을 대체할 수 있음(이 경우, 딕셔너리 형태로 해줘야함)

# 중복 데이터 집계 함수를 이용한 pivot table 만들기
df = df.pivot_table(values = ['강간(검거)','강간(발생)','강도(검거)','강도(발생)','살인(검거)','살인(발생)','소계(검거)','소계(발생)','절도(검거)','절도(발생)','폭력(검거)','폭력(발생)'], 
                    index = ['구 별'], aggfunc = 'sum')  # df.pivot_table(values = [집계할 대상(=columns)], index = [행의 기준으로 삼을 대상], aggfunc = '열의 데이터값을 계산해줄 함수')

# drop 을 사용하여 NaN 값이 포함된 행 제외하기
df = df.drop('구 없음')  # df.drop('없애고 싶은 특정 값(ex. NaN) 지정' 혹은 '삭제하려는 행 또는 열의 이름', axis = 0 혹은 1(0은 행, 1은 열))

# 검거율 구한 후 column 추가하기
df['강간검거율'] = (df['강간(검거)'] / df['강간(발생)']) * 100  # 백분율 구하는 거니까 먼저 나눈 값에다가 100 곱해주기
df['강도검거율'] = (df['강도(검거)'] / df['강도(발생)']) * 100
df['살인검거율'] = (df['살인(검거)'] / df['살인(발생)']) * 100
df['절도검거율'] = (df['절도(검거)'] / df['절도(발생)']) * 100
df['폭력검거율'] = (df['폭력(검거)'] / df['폭력(발생)']) * 100
df['검거율'] = (df['소계(검거)'] / df['소계(발생)']) * 100

# del 을 사용하여 컬럼 삭제하기
del df['강간(검거)']  # 예시에는 del df.['컬럼명'] 하고 나와있던데 . 쓰면 오류나던데????
del df['강도(검거)']
del df['살인(검거)']
del df['절도(검거)']
del df['폭력(검거)']
del df['소계(검거)']
del df['소계(발생)']

# rename 을 사용하여 컬럼명 변경해주기
df = df.rename(columns = {'강간(발생)':'강간','강도(발생)':'강도','살인(발생)':'살인','절도(발생)':'절도','폭력(발생)':'폭력'})  # df.rename(columns = {딕셔너리})바꾸고 싶은 컬럼의 이름을 딕셔너리 형태로 할당해준다

df


,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,검거율
구 별,,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594,66.519670
강동구,156,6,4,2366,2712,78.846154,133.333333,75.000000,33.347422,82.890855,60.469108
강북구,153,14,7,1434,2649,82.352941,92.857143,114.285714,43.096234,88.637222,73.126615
강서구,262,13,7,2096,3207,72.900763,100.000000,114.285714,60.114504,84.752105,75.022381
관악구,320,12,9,2706,3298,69.062500,116.666667,88.888889,30.561715,80.109157,58.502758
광진구,240,14,4,3026,2625,91.666667,185.714286,100.000000,42.200925,83.047619,62.734811
구로구,281,15,8,2335,3007,58.362989,73.333333,75.000000,38.072805,80.877951,62.026213
금천구,151,6,3,1567,2054,80.794702,100.000000,133.333333,56.668794,86.465433,73.948691
노원구,197,7,10,2193,2723,61.421320,100.000000,100.000000,36.525308,85.530665,63.703704


In [74]:

# 불러온 엑셀 파일을 바탕으로 DataFrame 출력하기
data = {'관서명':['서울', '부산'], '살인 발생':[10, 8], '살인 검거':[10, 8], '강도 발생':[20, 15], '강도 검거':[19, 14], '강간 발생':[30, 25], '강간 검거':[28, 23], 
        '절도 발생':[40, 35], '절도 검거':[35, 30], '폭력 발생':[50, 45], '폭력 검거':[48, 43]}  # 딕셔너리 형태로 데이터를 data 변수에 할당해줌
df = pd.DataFrame(data)  # DataFrame 을 사용하여 해당 변수로 데이터프레임을 출력해준다, 반드시 대소문자 구분할 것
df

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,서울,10,10,20,19,30,28,40,35,50,48
1,부산,8,8,15,14,25,23,35,30,45,43
